# 組み合わせ最適化
組み合わせ最適化は、基本的に説明変数が整数値をとり最適化を行います。説明変数は0または1をとるときも多いです。典型的な例でナップザック問題をやってみます。

ナップザック問題では、  
$N = \{1,2, \cdots , n\}$個の品物、容量 $c (c > 0)$が与えられたとき  
品物$i \in N$の重さを$w_i$、価値を$v_i$とすると、  

目的関数 (最大化)： $\sum_{i = 1}^n v_i x_i$  
制約条件 : $\sum_{i = 1}^n w_i x_i \leq c$  
$x_i \in \{0, 1\}, \forall i \in N$

となります。  
参考：今日から使える!組み合わせ最適化 離散問題ガイドブック 講談社　

### 例題
Pythonで始めるプログラミング入門 コロナ社 P127 プログラム10-2 を改変 

In [1]:
class KnapsackClass:

  def __init__(self, name, weight, value, capacity):
    self.name = name
    self.weight = weight
    self.value = value
    self.capacity = capacity
    self.all_list = [i for i in range(len(self.name))] #品物の要素番号

  def getValue(self, cur_list): #現在リストの価値を返す
    total = 0
    for i in cur_list:
      total += self.value[i]
    return total

  def getWeight(self, cur_list): #リストの重さを返す
    total = 0
    for i in cur_list:
      total += self.weight[i]
    return total

  def getBestlist(self, l): #再帰呼び出しで最適なリストを作成
    max_value = 0
    bestl = l
    for i in self.all_list:
      if i in l: #重複を許さない
        continue
      if self.getWeight(l + [i]) > self.capacity:
        continue
      rl = self.getBestlist(l + [i]) #要素を追加
      if max_value < self.getValue(rl):
        max_value = self.getValue(rl)
        bestl = rl
    return bestl

  def getResult(self, l):
    value = 0
    weight = 0
    combi = []
    for i in l:
      value += self.value[i]
      weight += self.weight[i]
      combi.append(self.name[i])
    return value, weight, combi

if __name__ == '__main__':
  name = ['チョコ', 'ポテトチップス', 'クッキー', 'ラムネ', 'ガム']
  weight = [130, 120, 80, 30, 20]
  value = [18, 15, 12, 4, 2]
  capacity = 300
  knap = KnapsackClass(name, weight, value, capacity)
  bestlist = knap.getBestlist([])
  #結果表示
  value, weight, combi = knap.getResult(bestlist)
  print('価値 = {0}, 重さ = {1}'.format(value, weight))
  print('組み合わせ : {0}'.format(combi))

価値 = 39, 重さ = 300
組み合わせ : ['チョコ', 'ポテトチップス', 'ラムネ', 'ガム']


## ソルバーを使ったナップザック問題

In [2]:
#組み合わせ最適化用ライブラリのインストール
!pip install pulp
!pip install ortoolpy

In [3]:
#商品の重複を許していない場合
from ortoolpy import knapsack
name = ['チョコ', 'ポテトチップス', 'クッキー', 'ラムネ', 'ガム']
price = [130, 120, 80, 30, 20]
like = [18, 15, 12, 4, 2]
capacity = 300
result = knapsack(price, like, capacity)
print('満足度 = {0}'.format(result[0]))
weight = 0
for i in result[1]:
  print(name[i])
  weight += price[i]
print('値段 = {0}'.format(weight))

満足度 = 39.0
チョコ
ポテトチップス
ラムネ
ガム
値段 = 300


## deapを使った遺伝アルゴリズムでの組み合わせ最適化
https://dse-souken.com/2021/05/25/ai-19/

In [4]:
!pip install deap

In [5]:
import random
import numpy
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

class KnapSackDeap:
  def __init__(self, name, weight, value, capacity, NGEN, POP, CXPB, MUTPB):
    self.name = name
    self.weight = weight
    self.value = value
    self.capacity = capacity
    self.NGEN = NGEN #世代数 (何世代計算するか)
    self.POP = POP #個体数 (1世代の個体数)
    self.CXPB = CXPB #交叉確率
    self.MUTPB = MUTPB #突然変異確率
    self.toolbox = base.Toolbox()

    #GAモデルの設定
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    #creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0)) 
    #2つある適応度の1つを最大化、もう1つを最小化する場合は、weights=(1.0,-1.0)と書く
    #最小化の場合 creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    #最大化の場合 creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    #creator.create("Individual", set, fitness=creator.Fitness) #個体の定義
    # creator.create("Individual", list, fitness=creator.FitnessMin)
    # creator.create("Individual", list, fitness=creator.FitnessMax)

    #アイテム辞書の作成
    self.items = {}
    for i in range(len(weight)):
      self.items[i] = (weight[i], value[i])
    random.seed(64) #乱数の種を初期化

    #GAの設定
    self.setGA()

  def evalKnapsack(self, individual):#目的関数
    weight = 0.0
    value = 0.0
    for i in range(len(self.items)):
        weight += self.items[i][0] * individual[i]
        value += self.items[i][1] * individual[i] 
    if weight > self.capacity:
        value=0.0
    return value,

  def setGA(self):
    self.toolbox.register("attribute", random.randint, 0,1) #random.randintの別名を「attribute」として定義, 0,1の範囲で発生
    self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attribute, len(self.items)) #individualの定義、len(items)=5なので、遺伝子の長さは5
    self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual) #populationの定義, listで遺伝子を渡す
    self.toolbox.register("select", tools.selTournament, tournsize=5) #選択の定義, tournsizeは当面5 
    self.toolbox.register("mate", tools.cxOnePoint) #交叉関数、今回は一点交叉
    self.toolbox.register("mutate", tools.mutUniformInt,low=0,up=1,indpb=0.2) #突然変異、変異は 0~1の整数で変異、各遺伝子の突然変異の確率は0.2
    self.toolbox.register("evaluate", self.evalKnapsack) #目的関数
    self.pop = self.toolbox.population(n=self.POP) #集団数を変数に入れておく

  def getGA(self):
    for ind in self.pop: #集団内の個体の適応度（目的関数の値）を計算
      ind.fitness.values = self.toolbox.evaluate(ind)
    hof = tools.ParetoFront() #パレート曲線上の個体(良い結果の個体)をhofに格納
    flow = algorithms.eaSimple(self.pop, self.toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, halloffame=hof) #Simple GAを利用
    best_ind = tools.selBest(self.pop, 1)[0] #最終的な集団(pop)からベストな個体を1体選出する関数
    return best_ind

if __name__ == '__main__':
  name = ['チョコ', 'ポテトチップス', 'クッキー', 'ラムネ', 'ガム']
  weight = [130, 120, 80, 30, 20]
  value = [18, 15, 12, 4, 2]
  capacity = 300
  NGEN = 50 #遺伝子数
  POP = 80 #世代数
  CXPB = 0.9 #交叉確率
  MUTPB = 0.1 #突然変異確率
  knap = KnapSackDeap(name, weight, value, capacity, NGEN, POP, CXPB, MUTPB)
  best_ind = knap.getGA()
  print('Best Gene : {0}, Objective = {1}'.format(best_ind, best_ind.fitness.values))
  for i, val in enumerate(best_ind):
    if val > 0:
      print(name[i])

gen	nevals
0  	0     
1  	72    
2  	70    
3  	73    
4  	70    
5  	72    
6  	74    
7  	73    
8  	70    
9  	69    
10 	76    
11 	76    
12 	72    
13 	78    
14 	77    
15 	74    
16 	74    
17 	73    
18 	76    
19 	78    
20 	74    
21 	74    
22 	71    
23 	72    
24 	68    
25 	76    
26 	75    
27 	69    
28 	74    
29 	73    
30 	74    
31 	72    
32 	78    
33 	74    
34 	64    
35 	74    
36 	74    
37 	73    
38 	69    
39 	71    
40 	72    
41 	80    
42 	73    
43 	73    
44 	69    
45 	72    
46 	71    
47 	72    
48 	73    
49 	73    
50 	78    
Best Gene : [1, 1, 0, 1, 1], Objective = (39.0,)
チョコ
ポテトチップス
ラムネ
ガム
